# Knowledge Structure Graph Generator

## 1. DB연결

In [1]:
import pymysql

In [2]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [3]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [5]:
import pandas as pd

result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
   DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0    소영_0      scs  서울,여자부,1.0\n현대캐피탈,안산,2.1\n승점,선두,2.35\n스코어,서울,2...
1   자연어_0      scs  의류,기능,1.91\n비서,아이언맨,2.35\n인공지능,비서,2.87\n의류,자체,...
2   자연어_4      scs  내년,도입,1.8\n활용,계획,2.76\n분석,차기,2.76\n교과서,차기,2.76...
3  카이스트_1      scs  바이러스,유전자,1.63\n대상,승인,2.1\n코로나,개발,2.67\n임상,대상,2...
4  카이스트_2      scs  건설,산업통상자원부,1.0\n북한,지역,1.76\n건설,지역,1.8\n산업통상자원부...
5  카이스트_3      scs  포트,나이트,1.0\n디지털,기술,2.7\n현실,가상,2.92\n모바일,비즈니스,2...
6  카이스트_5      scs  방사능,일침,1.31\n칼륨,유효,1.8\n월성원전,일침,2.1\n정도,관련,2.1...


In [6]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [10]:
file_name = "카이스트_1" 

#### 추출하기 원하는 인덱스 선정

In [11]:
condition = result['DOC_ID'] == file_name
result_content = result[condition]['KNOWLEDGE_STRUCTURE'].reset_index()

In [12]:
result_content['KNOWLEDGE_STRUCTURE'][0]

'바이러스,유전자,1.63\n대상,승인,2.1\n코로나,개발,2.67\n임상,대상,2.76\n승인,업계,2.76\n계획,업계,2.76\n효능,다국적,2.76\n고령자,평가,2.76\n고령자,확대,2.76\n백신,코로나,2.82\n백신,바이러스,3.0\n대상,평가,3.54\n개발,효능,4.0\n개발,지난해,4.0\n승인,지난해,4.0\n바이러스,기술,4.32\n백신,공동,4.44\n임상,강남,4.55\n'

In [13]:
# data = knowledge_structure[1] #인덱스만 변경하기
data = result_content['KNOWLEDGE_STRUCTURE'][0]
print(data)

바이러스,유전자,1.63
대상,승인,2.1
코로나,개발,2.67
임상,대상,2.76
승인,업계,2.76
계획,업계,2.76
효능,다국적,2.76
고령자,평가,2.76
고령자,확대,2.76
백신,코로나,2.82
백신,바이러스,3.0
대상,평가,3.54
개발,효능,4.0
개발,지난해,4.0
승인,지난해,4.0
바이러스,기술,4.32
백신,공동,4.44
임상,강남,4.55



In [14]:
# 7점 스케일을 0~1 범위로 바꿈
def similarity_normalize(num):
    value = (8 - float(num))/7
    return value

In [15]:
import re

contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [16]:
contents_normalize = []
for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    
    value[2] = round(similarity_normalize(value[2]), 3)
    new_value = value[0] +","+ value[1] +","+ str(value[2])
    
    contents_normalize.append(new_value)
    
    print(new_value)

바이러스,유전자,0.91
대상,승인,0.843
코로나,개발,0.761
임상,대상,0.749
승인,업계,0.749
계획,업계,0.749
효능,다국적,0.749
고령자,평가,0.749
고령자,확대,0.749
백신,코로나,0.74
백신,바이러스,0.714
대상,평가,0.637
개발,효능,0.571
개발,지난해,0.571
승인,지난해,0.571
바이러스,기술,0.526
백신,공동,0.509
임상,강남,0.493


## 3. Data JavaScript File 생성

### 파일 2개 생성
* keyword.js : data(nodes, edges)
* keyword.html : graph

### 01. keyword.js

In [41]:
import networkx as nx

In [42]:
G = nx.Graph()

In [43]:
# 에지리스트를 그래프에 넣기 : 그래프에 입력된 노드 확인용
for content in contents_normalize:
    tmp = re.split(',', content)
    G.add_edge(tmp[0], tmp[1], weight = tmp[2])

In [44]:
degree_list = [degree for degree in G.degree()] # 각 노드별 차수 저장

In [45]:
# 파일 저장용 nodes, edges
graph_nodes = "var nodes = "
graph_edges = "var edges = "

# 리스트 생성용
num = 0
nodes_list = []
reverse_graph = dict()

for name in G.nodes():
    # 딕셔너리 1개씩 생성하여 넣기
    tmp = dict()
    tmp['id'] = num
    tmp['label'] = name
    tmp['degree'] = degree_list[num][1]

    
    # 색상은 파랑색으로 지정
    tmp['color'] = "#97C2FC"
    
#     # 차수가 3이상이면 빨간색 지정
#     if(degree_list[num][1] >= 3):
#         tmp['color'] = "#FB7E81"
#     else:
#         tmp['color'] = "#97C2FC"
        
    nodes_list.append(tmp)   # {id: 0, label: 'hemp'}형태
    
    reverse_graph[name] = num   # {'hemp':0, 'cannabi':1}형태, 목적: 'hemp'검색시 바로 노드 번호 확인(편의성 목적)
    num += 1

In [46]:
nodes_list

[{'id': 0, 'label': '바이러스', 'degree': 3, 'color': '#97C2FC'},
 {'id': 1, 'label': '유전자', 'degree': 1, 'color': '#97C2FC'},
 {'id': 2, 'label': '대상', 'degree': 3, 'color': '#97C2FC'},
 {'id': 3, 'label': '승인', 'degree': 3, 'color': '#97C2FC'},
 {'id': 4, 'label': '코로나', 'degree': 2, 'color': '#97C2FC'},
 {'id': 5, 'label': '개발', 'degree': 3, 'color': '#97C2FC'},
 {'id': 6, 'label': '임상', 'degree': 2, 'color': '#97C2FC'},
 {'id': 7, 'label': '업계', 'degree': 2, 'color': '#97C2FC'},
 {'id': 8, 'label': '계획', 'degree': 1, 'color': '#97C2FC'},
 {'id': 9, 'label': '효능', 'degree': 2, 'color': '#97C2FC'},
 {'id': 10, 'label': '다국적', 'degree': 1, 'color': '#97C2FC'},
 {'id': 11, 'label': '고령자', 'degree': 2, 'color': '#97C2FC'},
 {'id': 12, 'label': '평가', 'degree': 2, 'color': '#97C2FC'},
 {'id': 13, 'label': '확대', 'degree': 1, 'color': '#97C2FC'},
 {'id': 14, 'label': '백신', 'degree': 3, 'color': '#97C2FC'},
 {'id': 15, 'label': '지난해', 'degree': 2, 'color': '#97C2FC'},
 {'id': 16, 'label': '기술', 

In [47]:
graph_nodes += str(nodes_list) + ";"
print(graph_nodes)

var nodes = [{'id': 0, 'label': '바이러스', 'degree': 3, 'color': '#97C2FC'}, {'id': 1, 'label': '유전자', 'degree': 1, 'color': '#97C2FC'}, {'id': 2, 'label': '대상', 'degree': 3, 'color': '#97C2FC'}, {'id': 3, 'label': '승인', 'degree': 3, 'color': '#97C2FC'}, {'id': 4, 'label': '코로나', 'degree': 2, 'color': '#97C2FC'}, {'id': 5, 'label': '개발', 'degree': 3, 'color': '#97C2FC'}, {'id': 6, 'label': '임상', 'degree': 2, 'color': '#97C2FC'}, {'id': 7, 'label': '업계', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '계획', 'degree': 1, 'color': '#97C2FC'}, {'id': 9, 'label': '효능', 'degree': 2, 'color': '#97C2FC'}, {'id': 10, 'label': '다국적', 'degree': 1, 'color': '#97C2FC'}, {'id': 11, 'label': '고령자', 'degree': 2, 'color': '#97C2FC'}, {'id': 12, 'label': '평가', 'degree': 2, 'color': '#97C2FC'}, {'id': 13, 'label': '확대', 'degree': 1, 'color': '#97C2FC'}, {'id': 14, 'label': '백신', 'degree': 3, 'color': '#97C2FC'}, {'id': 15, 'label': '지난해', 'degree': 2, 'color': '#97C2FC'}, {'id': 16, 'label': '기술', 'deg

In [48]:
print(reverse_graph) # graph_nodes를 뒤집어서 저장, 예를 들어  label명 : id명

{'바이러스': 0, '유전자': 1, '대상': 2, '승인': 3, '코로나': 4, '개발': 5, '임상': 6, '업계': 7, '계획': 8, '효능': 9, '다국적': 10, '고령자': 11, '평가': 12, '확대': 13, '백신': 14, '지난해': 15, '기술': 16, '공동': 17, '강남': 18}


In [49]:
edges_list = []

edge_max = 0.0
edge_min = 1.0

for edge in G.edges(data='weight'):
#     print("step: " + edge[2])
    tmp = float(edge[2])

    #최댓값 저장
    if(edge_max <= tmp):
        edge_max = tmp
    
    #최솟값 저장
    if(edge_min >= tmp):
        edge_min = tmp
        
#     print("edge_max: " + str(edge_max))
#     print("edge_min: " + str(edge_min))
#     print()

In [50]:
# edges는 {'hemp', 'cannabi', '3.21'}형태
for edge in G.edges(data='weight'):
    tmp = dict()
    tmp['from']   = reverse_graph.get(edge[0])     # 'hemp'검색시 인덱스값
    tmp['to']     = reverse_graph.get(edge[1])     # 'cannabi'검색시 인덱스값
    tmp['label']  = edge[2]                        # 가중치 저장
    
    # 최댓값 범위
    scale_max_value = 1.0
    length = float(edge[2])
    
#     convert_edge_min = scale_max_value - edge_max
#     minmax_scale = ((length - convert_edge_min) / (edge_max - edge_min)) * 500        # 길이를 min max 스케일링
#     tmp['value']  = round(minmax_scale, 2)         # Edge 두께를 가중치의 크기에 따라 변경(숫자가 클수록 두껍게)
  
    length = scale_max_value - float(edge[2])
    tmp['length'] = round(length * 800 + 100, 2)    # Edge 길이를 min max 스케일링
    
    tmp['color'] = "#d3d3d3"                      

    edges_list.append(tmp)

In [51]:
graph_edges += str(edges_list) + ";"
print(graph_edges)

var edges = [{'from': 0, 'to': 1, 'label': '0.91', 'length': 172.0, 'color': '#d3d3d3'}, {'from': 0, 'to': 14, 'label': '0.714', 'length': 328.8, 'color': '#d3d3d3'}, {'from': 0, 'to': 16, 'label': '0.526', 'length': 479.2, 'color': '#d3d3d3'}, {'from': 2, 'to': 3, 'label': '0.843', 'length': 225.6, 'color': '#d3d3d3'}, {'from': 2, 'to': 6, 'label': '0.749', 'length': 300.8, 'color': '#d3d3d3'}, {'from': 2, 'to': 12, 'label': '0.637', 'length': 390.4, 'color': '#d3d3d3'}, {'from': 3, 'to': 7, 'label': '0.749', 'length': 300.8, 'color': '#d3d3d3'}, {'from': 3, 'to': 15, 'label': '0.571', 'length': 443.2, 'color': '#d3d3d3'}, {'from': 4, 'to': 5, 'label': '0.761', 'length': 291.2, 'color': '#d3d3d3'}, {'from': 4, 'to': 14, 'label': '0.74', 'length': 308.0, 'color': '#d3d3d3'}, {'from': 5, 'to': 9, 'label': '0.571', 'length': 443.2, 'color': '#d3d3d3'}, {'from': 5, 'to': 15, 'label': '0.571', 'length': 443.2, 'color': '#d3d3d3'}, {'from': 6, 'to': 18, 'label': '0.493', 'length': 505.6, 'c

In [52]:
graph_data = graph_nodes + "\n\n" + graph_edges

In [53]:
print(graph_data)

var nodes = [{'id': 0, 'label': '바이러스', 'degree': 3, 'color': '#97C2FC'}, {'id': 1, 'label': '유전자', 'degree': 1, 'color': '#97C2FC'}, {'id': 2, 'label': '대상', 'degree': 3, 'color': '#97C2FC'}, {'id': 3, 'label': '승인', 'degree': 3, 'color': '#97C2FC'}, {'id': 4, 'label': '코로나', 'degree': 2, 'color': '#97C2FC'}, {'id': 5, 'label': '개발', 'degree': 3, 'color': '#97C2FC'}, {'id': 6, 'label': '임상', 'degree': 2, 'color': '#97C2FC'}, {'id': 7, 'label': '업계', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '계획', 'degree': 1, 'color': '#97C2FC'}, {'id': 9, 'label': '효능', 'degree': 2, 'color': '#97C2FC'}, {'id': 10, 'label': '다국적', 'degree': 1, 'color': '#97C2FC'}, {'id': 11, 'label': '고령자', 'degree': 2, 'color': '#97C2FC'}, {'id': 12, 'label': '평가', 'degree': 2, 'color': '#97C2FC'}, {'id': 13, 'label': '확대', 'degree': 1, 'color': '#97C2FC'}, {'id': 14, 'label': '백신', 'degree': 3, 'color': '#97C2FC'}, {'id': 15, 'label': '지난해', 'degree': 2, 'color': '#97C2FC'}, {'id': 16, 'label': '기술', 'deg

In [54]:
f = open("./results/result_webgraph/data/" + file_name + ".js", 'w', encoding = 'utf8')
f.write(graph_data)
f.close()
print(file_name + ".js 파일 생성 완료")

카이스트_1.js 파일 생성 완료


### 2. keyword.html

In [55]:
html_top = """
<!doctype html>
<html>
<head>
	<title>Knowledge Structure</title>
    """

In [56]:
vis_location = """<script type="text/javascript" src="../../lib/drawing/vis.js"></script>
    """

In [57]:
js_location = """<script src="../../results/result_webgraph/data/""" + file_name + """.js" charset="utf-8">
    </script>
    <style type="text/css">#mynetwork 
        {width: 100%;height: 1000px;border: 1px solid lightgray;}
    </style>
</head>
<body>
    """

In [58]:
title = """<p> Cheonsol of Knowledge Structure.</p>"""

In [59]:
body = """
    <div id="mynetwork"></div>
	<script type="text/javascript">
		var container = document.getElementById('mynetwork');
		var data = { nodes: nodes, edges: edges }; """

In [60]:
option = """
        var options = 
        { "nodes": {"shape" : "circle",
                    "font":{"size": 35}},
        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, 
            "smooth": {"type":"continuous","roundness": 0.5}},
            "physics": {"barnesHut": {"centralGravity": 0.001},
            "minVelocity": 0.75}};
        """

In [61]:
bottom = """var network = new vis.Network(container, data, options);
	</script>
</body>
</html>
"""

In [62]:
html_text = html_top + vis_location + js_location + title + body + option + bottom

In [63]:
html_text

'\n<!doctype html>\n<html>\n<head>\n\t<title>Knowledge Structure</title>\n    <script type="text/javascript" src="../../lib/drawing/vis.js"></script>\n    <script src="../../results/result_webgraph/data/카이스트_1.js" charset="utf-8">\n    </script>\n    <style type="text/css">#mynetwork \n        {width: 100%;height: 1000px;border: 1px solid lightgray;}\n    </style>\n</head>\n<body>\n    <p> Cheonsol of Knowledge Structure.</p>\n    <div id="mynetwork"></div>\n\t<script type="text/javascript">\n\t\tvar container = document.getElementById(\'mynetwork\');\n\t\tvar data = { nodes: nodes, edges: edges }; \n        var options = \n        { "nodes": {"shape" : "circle",\n                    "font":{"size": 35}},\n        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, \n            "smooth": {"type":"continuous","roundness": 0.5}},\n            "physics": {"barnesHut": {"centralGravity": 0.001},\n            "minVelocity": 0.75}};\n        var network = new vis.Network(container, data,

In [64]:
html_file = open('./results/result_webgraph/' + file_name + '.html', 'w')
html_file.write(html_text)
html_file.close()
print(file_name + ".html 파일 생성 완료")

카이스트_1.html 파일 생성 완료
